## Project - Bank Churn Prediction

In [48]:
#Importing Tensorflow
import tensorflow as tf
print(tf.__version__)

2.3.0


In [49]:
#Importing libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve, auc
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers.experimental import preprocessing

####1. Read the dataset

In [50]:
#Mounting Google Drive
from google.colab import drive

In [51]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [52]:
project_path = '/content/drive/My Drive/Colab Notebooks/'

In [53]:
dataset = project_path + 'bank.csv'

In [54]:
dframe = pd.read_csv(dataset)

In [55]:
dframe.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


####2. Drop the columns which are unique for all users like IDs

In [56]:
#2. Drop the columns which are unique for all users like IDs
dframe = dframe.drop(["RowNumber","CustomerId","Surname", "Geography",	"Gender"], axis = 1)

In [57]:
# Checking new dataframe
dframe.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,42,2,0.00,1,1,1,101348.88,1
1,608,41,1,83807.86,1,0,1,112542.58,0
2,502,42,8,159660.80,3,1,0,113931.57,1
3,699,39,1,0.00,2,0,0,93826.63,0
4,850,43,2,125510.82,1,1,1,79084.10,0


In [58]:
dframe.describe()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


####3. Distinguish the feature and target set

In [59]:
# Specify the data 
X=dframe.iloc[:,4:7]

# Specifying target labels
y = dframe.iloc[:, -1]

####4. Divide the data set into training and test sets

In [60]:
#4. Divide the data set into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

y_train =  np.array(y_train)
y_test =  np.array(y_test)

####5. Normalize the train and test data

In [61]:
X=tf.keras.utils.normalize(
    X, axis=-1, order=2)

In [62]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(8000, 3)
(2000, 3)
(8000,)
(2000,)


####6. Initialize & build the model. Identify the points of improvement and implement the same.

In [63]:
model = Sequential()

In [64]:
model.add(Dense(32, input_shape = (32,3), activation = 'relu'))
model.add(Dense(32, activation = 'tanh'))
model.add(Dense(1, activation = 'sigmoid'))

###7. Predict the results using 0.5 as a threshold

In [65]:
sgd = optimizers.Adam(lr = 0.5)

In [66]:
model.compile(optimizer = sgd, loss = 'binary_crossentropy', metrics=['accuracy'])

In [67]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 32, 32)            128       
_________________________________________________________________
dense_4 (Dense)              (None, 32, 32)            1056      
_________________________________________________________________
dense_5 (Dense)              (None, 32, 1)             33        
Total params: 1,217
Trainable params: 1,217
Non-trainable params: 0
_________________________________________________________________


######To begin we will test the model with batch size=10 and epochs=10 and verify the accuracy

In [68]:
#Testing the model with batch_size = 10 and epochs = 10
model.fit(X_train, y_train, batch_size = 10, epochs = 10, verbose = 1)

Epoch 1/10
800/800 [==============================] - 1s 2ms/step - loss: 0.9926 - accuracy: 0.6977
Epoch 2/10
800/800 [==============================] - 2s 2ms/step - loss: 0.9179 - accuracy: 0.6967
Epoch 3/10
800/800 [==============================] - 1s 2ms/step - loss: 0.9564 - accuracy: 0.6940
Epoch 4/10
800/800 [==============================] - 1s 1ms/step - loss: 0.8136 - accuracy: 0.7075
Epoch 5/10
800/800 [==============================] - 1s 1ms/step - loss: 0.9329 - accuracy: 0.6980
Epoch 6/10
800/800 [==============================] - 1s 1ms/step - loss: 0.9045 - accuracy: 0.7088
Epoch 7/10
800/800 [==============================] - 1s 1ms/step - loss: 1.0075 - accuracy: 0.6988
Epoch 8/10
800/800 [==============================] - 1s 1ms/step - loss: 0.9388 - accuracy: 0.7030
Epoch 9/10
800/800 [==============================] - 1s 1ms/step - loss: 0.8605 - accuracy: 0.7078
Epoch 10/10
800/800 [==============================] - 1s 1ms/step - loss: 0.9436 - accuracy: 0.6995

###### We could see an accuracy for around 70% or 71%, we will now increase the number of batch size and epochs to see if there is an improvement on the performance of the model

In [69]:
#Testing the model with batch_size = 50 and epochs = 100
model.fit(X_train, y_train, batch_size = 50, epochs = 100, verbose = 1)

Epoch 1/100
160/160 [==============================] - 0s 2ms/step - loss: 0.6004 - accuracy: 0.7638
Epoch 2/100
160/160 [==============================] - 0s 1ms/step - loss: 0.5687 - accuracy: 0.7770
Epoch 3/100
160/160 [==============================] - 0s 1ms/step - loss: 0.6048 - accuracy: 0.7520
Epoch 4/100
160/160 [==============================] - 0s 2ms/step - loss: 0.5957 - accuracy: 0.7560
Epoch 5/100
160/160 [==============================] - 0s 2ms/step - loss: 0.6341 - accuracy: 0.7523
Epoch 6/100
160/160 [==============================] - 0s 1ms/step - loss: 0.6698 - accuracy: 0.7412
Epoch 7/100
160/160 [==============================] - 0s 1ms/step - loss: 0.5677 - accuracy: 0.7775
Epoch 8/100
160/160 [==============================] - 0s 2ms/step - loss: 0.6027 - accuracy: 0.7458
Epoch 9/100
160/160 [==============================] - 0s 2ms/step - loss: 0.6495 - accuracy: 0.7490
Epoch 10/100
160/160 [==============================] - 0s 2ms/step - loss: 0.6216 - accura

###### We can see now an improvement after incresing to batch size=50 and epochs=100. The accuracy of the model is now around 75% We will now increase the batch size and epochs to see if there is a new increase in the accuracy of the model

In [70]:
#Testing the model with batch_size = 100 and epochs = 500
model.fit(X_train, y_train, batch_size = 100, epochs = 500, verbose = 1)

Epoch 1/500
80/80 [==============================] - 0s 2ms/step - loss: 0.5563 - accuracy: 0.7715
Epoch 2/500
80/80 [==============================] - 0s 2ms/step - loss: 0.5437 - accuracy: 0.7945
Epoch 3/500
80/80 [==============================] - 0s 2ms/step - loss: 0.5452 - accuracy: 0.7880
Epoch 4/500
80/80 [==============================] - 0s 2ms/step - loss: 0.5855 - accuracy: 0.7670
Epoch 5/500
80/80 [==============================] - 0s 2ms/step - loss: 0.5391 - accuracy: 0.7945
Epoch 6/500
80/80 [==============================] - 0s 2ms/step - loss: 0.5779 - accuracy: 0.7645
Epoch 7/500
80/80 [==============================] - 0s 2ms/step - loss: 0.6059 - accuracy: 0.7515
Epoch 8/500
80/80 [==============================] - 0s 2ms/step - loss: 0.6113 - accuracy: 0.7615
Epoch 9/500
80/80 [==============================] - 0s 2ms/step - loss: 0.6565 - accuracy: 0.7372
Epoch 10/500
80/80 [==============================] - 0s 2ms/step - loss: 0.6957 - accuracy: 0.7310
Epoch 11/

##### It seems like there is no significant improvement in the model if we increase to batch size = 100 and epochs = 500, the accuracy remains at around 75%. We will now verify the accuracy of the model and print the confusion matrix. Hence, we will remain at batch size = 50 and epochs = 100

In [71]:
#Testing the model with batch_size = 50 and epochs = 100
model.fit(X_train, y_train, batch_size = 50, epochs = 100, verbose = 1)

Epoch 1/100
160/160 [==============================] - 0s 2ms/step - loss: 0.6677 - accuracy: 0.7170
Epoch 2/100
160/160 [==============================] - 0s 2ms/step - loss: 0.7099 - accuracy: 0.7145
Epoch 3/100
160/160 [==============================] - 0s 2ms/step - loss: 0.6203 - accuracy: 0.7582
Epoch 4/100
160/160 [==============================] - 0s 2ms/step - loss: 0.7075 - accuracy: 0.7220
Epoch 5/100
160/160 [==============================] - 0s 2ms/step - loss: 0.6403 - accuracy: 0.7415
Epoch 6/100
160/160 [==============================] - 0s 2ms/step - loss: 0.6310 - accuracy: 0.7490
Epoch 7/100
160/160 [==============================] - 0s 2ms/step - loss: 0.6528 - accuracy: 0.7260
Epoch 8/100
160/160 [==============================] - 0s 2ms/step - loss: 0.5914 - accuracy: 0.7728
Epoch 9/100
160/160 [==============================] - 0s 2ms/step - loss: 0.6792 - accuracy: 0.7250
Epoch 10/100
160/160 [==============================] - 0s 2ms/step - loss: 0.6106 - accura

#####8. Print the Accuracy score and confusion matrix

In [72]:
results = model.evaluate(X_test, y_test)

63/63 [==============================] - 0s 1ms/step - loss: 0.4986 - accuracy: 0.8035


In [73]:
print(model.metrics_names)
print(results)

['loss', 'accuracy']
[0.4986029267311096, 0.8034999966621399]


In [74]:
Y_pred_cls = model.predict_classes(X_test, batch_size=200, verbose=1)

Y_pred_cls = model.predict_classes(X_test, batch_size=200, verbose=0)
print('Accuracy Model1 (Dropout): '+ str(model.evaluate(X_test,y_test)[1]))
print('Recall_score: ' + str(recall_score(y_test,Y_pred_cls)))
print('Precision_score: ' + str(precision_score(y_test, Y_pred_cls)))
print('F-score: ' + str(f1_score(y_test,Y_pred_cls)))
print('Confusion Matrix: ')
confusion_matrix(y_test, Y_pred_cls)

63/63 [==============================] - 0s 1ms/step - loss: 0.4986 - accuracy: 0.8035
Accuracy Model1 (Dropout): 0.8034999966621399
Recall_score: 0.0
Precision_score: 0.0
F-score: 0.0
Confusion Matrix: 


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([[1607,    0],
       [ 393,    0]])

##**Conlusions:**

With this model we can predict with an accuracy of around 80% the customers that will leave in the next 6 months.
In general, the accuracy of the model remains at around 76% or 76% and sometimes even lower. A good set of parameters will be batch size = 50 and epochs = 100
There is no reason to increase the number of batchs and epochs as the performance remains the same.